In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub

In [ ]:
import kagglehub

# Download latest version

path = kagglehub.dataset_download("notlucasp/financial-news-headlines")

guardian = pd.read_csv(os.path.join(path, "guardian_headlines.csv"))
cnbc = pd.read_csv(os.path.join(path, "cnbc_headlines.csv"))
reuters = pd.read_csv(os.path.join(path, "reuters_headlines.csv"))

In [ ]:
#Convert dates to match
guardian["date"] = pd.to_datetime(guardian["Time"], format="%d-%b-%y", errors="coerce")
cnbc["Time"] = cnbc["Time"].astype(str).str.replace("ET", "", regex=False).str.strip()
cnbc["date"] = pd.to_datetime(cnbc["Time"], format="%I:%M %p %a, %d %B %Y", errors="coerce")
reuters["date"] = pd.to_datetime(reuters["Time"], format="%b %d %Y", errors="coerce")

# Add source column
guardian["source"] = "Guardian"
cnbc["source"] = "CNBC"
reuters["source"] = "Reuters"

# Make sure headlines match for merging
guardian.rename(columns={"Headlines": "headline"}, inplace=True)
cnbc.rename(columns={"Headlines": "headline"}, inplace=True)
reuters.rename(columns={"Headlines": "headline"}, inplace=True)

# make sure description exists and fill in null values
for df in [guardian, cnbc, reuters]:
    if "Description" not in df.columns:
        df["Description"] = None

# make headline and description all one
for df in [guardian, cnbc, reuters]:
    df["headline_description"] = df["headline"].fillna('') + " " + df["Description"].fillna('')

# --- Keep only the columns we want ---
guardian = guardian[["date", "headline_description", "source"]]
cnbc = cnbc[["date", "headline_description", "source"]]
reuters = reuters[["date", "headline_description", "source"]]

# --- Combine all datasets ---
combined_df = pd.concat([guardian, cnbc, reuters], ignore_index=True)

# --- Drop rows with missing date or text, and sort by date ---
combined_df = combined_df.dropna(subset=["headline_description", "date"])
combined_df = combined_df.sort_values("date").reset_index(drop=True)

# --- Preview ---
print("✅ Combined dataset shape:", combined_df.shape)
print(combined_df.head())


In [ ]:
print("Earliest date:", combined_df["date"].min())
print("Latest date:", combined_df["date"].max())

In [ ]:
# SEPERATING OUT BY SPECIFIC TIME PERIODS, using from late 2017 to late 2019 as to not include data about covid

# Define start and end of the period
start_date = pd.to_datetime("2017-12-18")
end_date = pd.to_datetime("2020-7-17")

# Filter by date range
filtered_df = combined_df[
    (combined_df["date"] >= start_date) &
    (combined_df["date"] <= end_date)
]

# Clean the headline text
filtered_df["headline_description"] = filtered_df["headline_description"].str.replace(r'\s+', ' ', regex=True).str.strip()


# making sure headline does not have any extra info

filtered_df["headline_description"] = filtered_df["headline_description"].str.replace(r'\s+', ' ', regex=True).str.strip()

print(filtered_df.head())
print(filtered_df.shape)
print(combined_df.shape)

In [ ]:
from google.colab import files
xl_market = files.upload()

Saving chart_20251124T231933.csv to chart_20251124T231933 (1).csv


In [ ]:
df = pd.read_csv("chart_20251124T231933.csv")

In [ ]:
df.head(
)

,Date,Value
0,11/24/2015,2089.14
1,11/25/2015,2088.87
2,11/27/2015,2090.11
3,11/30/2015,2080.41
4,12/01/2015,2102.63


In [ ]:
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")

# If you want the column literally as a string in YYYY-MM-DD format:
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

print(df.head())

         Date    Value
0  2015-11-24  2089.14
1  2015-11-25  2088.87
2  2015-11-27  2090.11
3  2015-11-30  2080.41
4  2015-12-01  2102.63


In [ ]:
df.rename(columns={"Date": "date"}, inplace=True)

In [ ]:
df.head(16000)


,date,Value
0,2015-11-24,2089.14
1,2015-11-25,2088.87
2,2015-11-27,2090.11
3,2015-11-30,2080.41
4,2015-12-01,2102.63
...,...,...
2510,2025-11-17,6672.41
2511,2025-11-18,6617.32
2512,2025-11-19,6642.16
2513,2025-11-20,6538.76


In [ ]:
df['date'] = pd.to_datetime(df['date'])

# Set 'date' as index for easier reindexing
df.set_index('date', inplace=True)

# Create a complete date range from the min to max date in df
full_date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')

# Reindex the DataFrame to include all dates in the range
df = df.reindex(full_date_range)

# Fill missing 'Value' with the previous valid observation
df['Value'] = df['Value'].fillna(method='ffill')

# Reset index to turn 'date' back into a column
df.reset_index(inplace=True)
df.rename(columns={'index': 'date'}, inplace=True)

print("DataFrame 'df' after filling missing dates and values:")
print(df.head())
print(df.tail())
print(f"New shape of df: {df.shape}")

DataFrame 'df' after filling missing dates and values:
        date    Value
0 2015-11-24  2089.14
1 2015-11-25  2088.87
2 2015-11-26  2088.87
3 2015-11-27  2090.11
4 2015-11-28  2090.11
           date    Value
3646 2025-11-17  6672.41
3647 2025-11-18  6617.32
3648 2025-11-19  6642.16
3649 2025-11-20  6538.76
3650 2025-11-21  6602.99
New shape of df: (3651, 2)


/tmp/ipython-input-3580574826.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Value'] = df['Value'].fillna(method='ffill')


In [ ]:
df["date"] = pd.to_datetime(df["date"])
new_combined_df = pd.merge(df, filtered_df, on="date", how="inner")
display(new_combined_df.head())

,date,Value,headline_description,source
0,2017-12-18,2690.16,Business live Bitcoin bubble warnings grow lou...,Guardian
1,2017-12-18,2690.16,Christmas tree cutters' labor fight shines lig...,Guardian
2,2017-12-18,2690.16,How the American government slowly became a bu...,Guardian
3,2017-12-18,2690.16,‘They stole the soul of the city’: how Warsaw'...,Guardian
4,2017-12-18,2690.16,"Desperate UK homeowners are cutting prices, sa...",Guardian


In [ ]:
new_combined_df.head()

,date,Value,headline_description,source
0,2017-12-18,2690.16,Business live Bitcoin bubble warnings grow lou...,Guardian
1,2017-12-18,2690.16,Christmas tree cutters' labor fight shines lig...,Guardian
2,2017-12-18,2690.16,How the American government slowly became a bu...,Guardian
3,2017-12-18,2690.16,‘They stole the soul of the city’: how Warsaw'...,Guardian
4,2017-12-18,2690.16,"Desperate UK homeowners are cutting prices, sa...",Guardian


In [ ]:
print("Earliest date:", combined_df["date"].min())
print("Latest date:", combined_df["date"].max())

Earliest date: 2017-12-17 00:00:00
Latest date: 2020-07-18 00:00:00


In [ ]:
print("Earliest date:", df["date"].min())
print("Latest date:", df["date"].max())

Earliest date: 2015-11-24 00:00:00
Latest date: 2025-11-21 00:00:00


In [ ]:
print("Earliest date:", df["date"].min())
print("Latest date:", df["date"].max())

Earliest date: 2015-11-24 00:00:00
Latest date: 2025-11-21 00:00:00


In [ ]:
print("Null values before dropping:")
print(new_combined_df.isnull().sum())
print(f"Shape before dropping: {new_combined_df.shape}")

new_combined_df.dropna(inplace=True)

print("\nNull values after dropping:")
print(new_combined_df.isnull().sum())
print(f"Shape after dropping: {new_combined_df.shape}")

Null values before dropping:
date                    0
Value                   0
headline_description    0
source                  0
dtype: int64
Shape before dropping: (50486, 4)

Null values after dropping:
date                    0
Value                   0
headline_description    0
source                  0
dtype: int64
Shape after dropping: (50486, 4)


In [ ]:
new_combined_df.head(200)

,date,Value,headline_description,source
0,2017-12-18,2690.16,Business live Bitcoin bubble warnings grow lou...,Guardian
1,2017-12-18,2690.16,Christmas tree cutters' labor fight shines lig...,Guardian
2,2017-12-18,2690.16,How the American government slowly became a bu...,Guardian
3,2017-12-18,2690.16,‘They stole the soul of the city’: how Warsaw'...,Guardian
4,2017-12-18,2690.16,"Desperate UK homeowners are cutting prices, sa...",Guardian
...,...,...,...,...
195,2017-12-30,2673.61,Crown Resorts agrees to $150m sale of CrownBet,Guardian
196,2017-12-30,2673.61,The long read American reams: why a ‘paperless...,Guardian
197,2017-12-30,2673.61,First-ever shipment of Russian gas unlikely to...,Guardian
198,2017-12-30,2673.61,Rail passengers lost 3.6m hours in delays in 2...,Guardian


In [ ]:
print("Earliest date:", new_combined_df["date"].min())
print("Latest date:", new_combined_df["date"].max())

Earliest date: 2017-12-18 00:00:00
Latest date: 2020-07-17 00:00:00


In [ ]:
unique_dates_count = new_combined_df['date'].nunique()
print(f"Number of unique date values in new_combined_df: {unique_dates_count}")

Number of unique date values in new_combined_df: 925


In [ ]:
import pandas as pd

# Add a future_date column to new_combined_df
new_combined_df['future_date'] = new_combined_df['date'] + pd.Timedelta(days=10)

# Prepare df for merge with only 'date' and 'Value'
df_for_merge = df[['date', 'Value']]

# Perform the merge to bring in the future market value
new_combined_df = pd.merge(
    new_combined_df,
    df_for_merge,
    left_on='future_date',
    right_on='date',
    how='left',
    suffixes=('_current', '_future') # Suffixes for overlapping column names
)

# Rename columns for clarity
new_combined_df.rename(columns={
    'date_current': 'date',
    'Value_current': 'Value',
    'Value_future': 'future_value'
}, inplace=True)

# Drop redundant columns created during the merge
new_combined_df.drop(columns=['date_future', 'future_date'], inplace=True)

# Delete null values in the 'future_value' column
print("Null values before dropping for 'future_value':")
print(new_combined_df['future_value'].isnull().sum())
print(f"Shape before dropping: {new_combined_df.shape}")

new_combined_df.dropna(subset=['future_value'], inplace=True)

print("\nNull values after dropping for 'future_value':")
print(new_combined_df['future_value'].isnull().sum())
print(f"Shape after dropping: {new_combined_df.shape}")

print("\nUpdated new_combined_df head:")
print(new_combined_df.head())

Null values before dropping for 'future_value':
0
Shape before dropping: (50486, 5)

Null values after dropping for 'future_value':
0
Shape after dropping: (50486, 5)

Updated new_combined_df head:
        date    Value                               headline_description  \
0 2017-12-18  2690.16  Business live Bitcoin bubble warnings grow lou...   
1 2017-12-18  2690.16  Christmas tree cutters' labor fight shines lig...   
2 2017-12-18  2690.16  How the American government slowly became a bu...   
3 2017-12-18  2690.16  ‘They stole the soul of the city’: how Warsaw'...   
4 2017-12-18  2690.16  Desperate UK homeowners are cutting prices, sa...   

     source  future_value  
0  Guardian       2687.54  
1  Guardian       2687.54  
2  Guardian       2687.54  
3  Guardian       2687.54  
4  Guardian       2687.54  


In [ ]:
unique_dates_count = new_combined_df['date'].nunique()
print(f"Number of unique date values in new_combined_df: {unique_dates_count}")

Number of unique date values in new_combined_df: 925


In [ ]:
new_combined_df.to_csv("new_combined_df.csv", index=False)
print("DataFrame saved to new_combined_df.csv")

DataFrame saved to new_combined_df.csv
